<html lang="de">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <style>
        h1 {
            font-size: 42px;
            font-weight: bold;
        }
    </style>
</head>
<body>
    <h1>Studienarbeit: Predictive Analytics</h1>
</body>
</html>


<h1>Laden der Datensätze: </h1>
Zum bearbeiten der Daten wird eine Stichprobe von von 300 Tagen zufällig von jedem Tag eines Jahres geladen.

In [60]:
import os

year = 2023

# Laden des Verzeichnisses
path = f'C:\\Users\\Marc Willberg\\Studienarbeit_Predictive_Analytics\\Data_Taxi_Trip_NY'
year_directory = os.path.join(path, f'{year}')

# Ausgabe aller gefundenen .parquet-Dateien
print("Folgende Daten wurden gefunden:")
parquet_files = [os.path.join(year_directory, file) for file in os.listdir(year_directory) if file.endswith('.parquet')]
for file in parquet_files:
    print(file)

Folgende Daten wurden gefunden:
C:\Users\Marc Willberg\Studienarbeit_Predictive_Analytics\Data_Taxi_Trip_NY\2023\yellow_tripdata_2023-01.parquet
C:\Users\Marc Willberg\Studienarbeit_Predictive_Analytics\Data_Taxi_Trip_NY\2023\yellow_tripdata_2023-02.parquet
C:\Users\Marc Willberg\Studienarbeit_Predictive_Analytics\Data_Taxi_Trip_NY\2023\yellow_tripdata_2023-03.parquet
C:\Users\Marc Willberg\Studienarbeit_Predictive_Analytics\Data_Taxi_Trip_NY\2023\yellow_tripdata_2023-04.parquet
C:\Users\Marc Willberg\Studienarbeit_Predictive_Analytics\Data_Taxi_Trip_NY\2023\yellow_tripdata_2023-05.parquet
C:\Users\Marc Willberg\Studienarbeit_Predictive_Analytics\Data_Taxi_Trip_NY\2023\yellow_tripdata_2023-06.parquet
C:\Users\Marc Willberg\Studienarbeit_Predictive_Analytics\Data_Taxi_Trip_NY\2023\yellow_tripdata_2023-07.parquet
C:\Users\Marc Willberg\Studienarbeit_Predictive_Analytics\Data_Taxi_Trip_NY\2023\yellow_tripdata_2023-08.parquet
C:\Users\Marc Willberg\Studienarbeit_Predictive_Analytics\Data_T


<h2>Einsicht auf Rohdaten:</h2>
<p>Für einen Einblick der Rohdaten werden aus der ersten Datei die ersten 5 Spalten geladen.</p>
<p>Erklärung der Datensätze: <a href="https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf" target="_blank">Data Dictionary – Yellow Taxi Trip Records</a></p>



In [53]:
from IPython.display import display

# Laden der ersten .parquet-Datei
first_file = parquet_files[0]
df = pd.read_parquet(first_file)

# Anzeige der ersten 5 Spalten
df_head = df.iloc[:5, :].T
display(df_head)

,0,1,2,3,4
VendorID,2,2,2,1,2
tpep_pickup_datetime,2023-01-01 00:32:10,2023-01-01 00:55:08,2023-01-01 00:25:04,2023-01-01 00:03:48,2023-01-01 00:10:29
tpep_dropoff_datetime,2023-01-01 00:40:36,2023-01-01 01:01:27,2023-01-01 00:37:49,2023-01-01 00:13:25,2023-01-01 00:21:19
passenger_count,1.0,1.0,1.0,0.0,1.0
trip_distance,0.97,1.1,2.51,1.9,1.43
RatecodeID,1.0,1.0,1.0,1.0,1.0
store_and_fwd_flag,N,N,N,N,N
PULocationID,161,43,48,138,107
DOLocationID,141,237,238,7,79
payment_type,2,1,1,1,1


<h2>Vorbereitung der Daten:</h2>
Zunächst werden die Daten vorbereitet um Algorithmen auf diesen anzuwenden.

In [74]:
class TaxiTripFeatures:
    def __init__(self, travel_time,start_month, start_day, start_hour, weekday, passenger_count, trip_distance, PULocationID, DOLocationID, tip_amount, total_amount_without_tip):
        self.travel_time = travel_time                           #Fahrtzeit in Sekunden
        self.start_day = start_day                               #Tag in dem das Taxi gestartet hat
        self.start_hour = start_hour                             #Stunde in dem das Taxi gestartet hat
        self.weekday = weekday                                   #Wochentag als int in dem das Taxi gestartet hat
        self.passenger_count = passenger_count                   #Anzahl der Passagiere
        self.trip_distance = trip_distance                       #Fahrtdistanz in Meilen 
        self.PULocationID = PULocationID                         #StartID Standort
        self.DOLocationID = DOLocationID                         #ZielID Standort
        self.tip_amount = tip_amount                             #Trinkgeld
        self.total_amount_without_tip = total_amount_without_tip #Gesamtbetrag ohne Trinkgeld

<h2>Laden der Strichprobe:</h2>
Zum bearbeiten der Daten wird eine Stichprobe von von 300 Tagen zufällig von jedem Tag eines Jahres geladen.

<h3>Überprüfung der Stichprobe:</h3> Stichprobe wird auf Varianz und Divesität geprüft. Dies ist notwendig um Sicherzustellen ob die Stichprobe die gesamten Datenmengen gut abbildet.

In [78]:
print(all_features[1])